In [14]:
#courtesy of Lu
#Github: https://github.com/luzhixiu1996/MLE-Phi/blob/master/Code/MLE.py

import math
import numpy as np
from scipy import optimize
from scipy.optimize import minimize_scalar
from sklearn.preprocessing import minmax_scale
from statistics import mean, median
import pandas as pd

PATH = '/nfs/home/ababjac/MLE-Phi/Data/'

#deltaEtaFile=PATH+"Yeast/Scer_Selection.csv"
#deltaMFile=PATH+"Yeast/Scer_Mutation.csv"

#deltaEtaFile=PATH+"ecoli/Ecoli_Selection.csv"
#deltaMFile=PATH+"ecoli/Ecoli_Mutation.csv"

#deltaEtaFile=PATH+"C_Elegen/Crei_Selection.csv"
#deltaMFile=PATH+"C_Elegen/Crei_Mutation.csv"

#deltaEtaFile=PATH+"D.Melaganster/Drer_Selection.csv"
#deltaMFile=PATH+"D.Melaganster/Drer_Mutation.csv"

deltaEtaFile=PATH+"Arabidopsis/Atha_Selection.csv"
deltaMFile=PATH+"Arabidopsis/Atha_Mutation.csv"

#----------------------------UNMODIFIED----------------------------#
deltaEtaFile=open(deltaEtaFile)
lines=deltaEtaFile.readlines()
etaDict=dict()
for line in lines[1:]:
    splitList=line.split(",")
    if len(splitList[0])>0 and len(splitList[1])>0:
        etaDict[splitList[1]]=splitList[2]

deltaMFile=open(deltaMFile)
lines=deltaMFile.readlines()
mDict=dict()
for line in lines[1:]:
    splitList=line.split(",")
    if len(splitList[0])>0 and len(splitList[1])>0:
        mDict[splitList[1]]=splitList[2]


codontable = {
'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
}

inverseTable=  {'S': ['TCA', 'TCC', 'TCT', 'TCG', 'AGC', 'AGT'], 'N': ['AAT', 'AAC'], 'W': ['TGG'],
                'D': ['GAT', 'GAC'], 'E': ['GAA', 'GAG'], 'F': ['TTC', 'TTT'], 'P': ['CCG', 'CCT', 'CCA', 'CCC'],
              'T': ['ACT', 'ACG', 'ACC', 'ACA'], 'G': ['GGG', 'GGC', 'GGT', 'GGA'],
              'R': ['AGA', 'CGT', 'CGA', 'AGG', 'CGG', 'CGC'], 'L': ['TTG', 'CTC', 'TTA', 'CTT', 'CTA', 'CTG'],
              'V': ['GTC', 'GTG', 'GTA', 'GTT'], 'A': ['GCA', 'GCT', 'GCC', 'GCG'], '*': ['TGA', 'TAA', 'TAG'],
              'H': ['CAC', 'CAT'], 'I': ['ATC', 'ATA', 'ATT'], 'K': ['AAG', 'AAA'], 'Y': ['TAT', 'TAC'],
              'M': ['ATG'], 'C': ['TGC', 'TGT'], 'Q': ['CAG', 'CAA']}

synonymonDict={}
for key in inverseTable:
    valueList=inverseTable[key]
    for value in valueList:
        synonymonDict[value]=valueList


def loadSequence(sequence):
    startCodon="ATG"
    stopCodonList=["TAG","TAA","TGA"]
    codonList=[]
    i=0
    while(i<len(sequence)):
        codon=sequence[i:i+3]
        if len(codon)==3:
            codonList.append(codon)
        i+=3
    actualCodonList=[]
    started=False
    for codon in codonList:
        if codon in stopCodonList:
            break
        if started:
            actualCodonList.append(codon)
        if codon==startCodon:
            started=True
    codonList=actualCodonList
   # print "codon readed successful, the number of codon in this sequence is %d"%(len(codonList))
    return codonList

#this method removes sequence that cant be handled by mDict, etaDict and "TGG",
def parseSequence(sequence):
    i=0
    startCodon="ATG"
    stopCodonList=["TAG","TAA","TGA"]
    parsedSequence=""
    while(i<len(sequence)):
        codon=sequence[i:i+3]
        if len(codon)==3:
            if (codon in mDict and codon in etaDict) or (codon in startCodon) or (codon in stopCodonList) and ("TGG" not in codon):
                parsedSequence+=codon
        i+=3
    return parsedSequence


def cutSequence(seq):
    sequence=seq
    startCodon="ATG"
    stopCodonList=["TAG","TAA","TGA"]
    codonList=[]
    i=0
    while(i<len(sequence)):
        codon=sequence[i:i+3]
        if len(codon)==3:
            codonList.append(codon)
        i+=3
    actualCodonList=[]
    started=FalsewindowSize
    for codon in codonList:
        if codon in stopCodonList:
            break
        if started:
            actualCodonList.append(codon)
        if codon==startCodon:
            started=True
    codonList=actualCodonList
   # print "codon readed successful, the number of codon in this sequence is %d"%(len(codonList))
    return codonList


def roundList(lst):
    roundedList=[]
    decimalPlaces=6
    for i in lst:
        roundedList.append(round(i,decimalPlaces))
    return roundedList

phiDict=dict()
def method4(codonList):
    global mDict
    global etaDict
    global phiDict
    phiList=[]

    for codon in codonList:
        if codon in phiDict:
             phiList.append(phiDict[codon])
        else:
            maxprob=0.0
            selectedPhi=0.0
            rangeList=[]
            for i in range(1,101):
                rangeList.append(i/100.0)
            for phi in rangeList:
                if codon in mDict:
                    deltaM=float(mDict[codon])
                else:
                    deltaM=1.0
                if codon in etaDict:
                    deltaEta=float(etaDict[codon])
                else:
                    deltaEta=1.0
                global synonymonDict
                synoList=synonymonDict[codon]
                divisor=np.exp(-1.00*deltaM-(deltaEta*phi))
                dividant=0
                for syno in synoList:
                    if syno in mDict:
                        deltaM=float(mDict[syno])
                    else:
                        deltaM=1.0
                    if syno in etaDict:
                        deltaEta=float(etaDict[syno])
                    else:
                        deltaEta=1.0
                    tmp=np.exp((-1.00*deltaM)-(deltaEta*phi))
    #                print((-1.00*deltaM)-(deltaEta*phi))
    #                print (tmp)
                    dividant+=tmp
                if not dividant==0:
                    prob=divisor/dividant
                else:
                    prob=0

                if prob>maxprob:
                    maxprob=prob
                    selectedPhi=phi
            if not selectedPhi==0:
                phiDict[codon]=selectedPhi
                phiList.append(selectedPhi)
            else:
                phiList.append(0.001)
                print ("found 0")
    return phiList

def setWindow(inputList,size):
    windowList=[]
    windowSize=size
    cnt=-1

    while True:
        cnt+=1
        if cnt+windowSize>len(inputList):
            break
        selectedList=inputList[cnt:cnt+windowSize]
        sum=0
        for i in selectedList:
            sum+=float(i)
        average=sum/len(selectedList)
        windowList.append(average)
    return windowList

from scipy.stats.mstats import gmean


def cal_mle_windows(sequence,windowSize=10):
    codonList=loadSequence(sequence)
    phiList=method4(codonList)# df['Phi_list'] = np.array(my_windows)
    windowList=setWindow(phiList,windowSize)
    #print(len(codonList), len(phiList), len(windowList))
    return windowList


def calPhiForGene(sequence):
    codonList=loadSequence(sequence)
    phiList=method4(codonList)
    avg=gmean(phiList)
    return avg
#----------------------------UNMODIFIED----------------------------#

import matplotlib.pyplot as plt
import scipy.stats as stats


def make_histogram_data(data, xlabel, plt_title, bins=None, num_bins=10):
    if bins == None:
        bins = np.linspace(math.floor(min(data)), math.ceil(max(data)), num_bins)

    plt.hist(data, bins)
    plt.title(plt_title)
    plt.xlabel(xlabel)
    plt.ylabel('Count')
    plt.show()

def generate_mlphi_plot(mlphi_list, title, avgRRT=None):
    indexes = list(range(1, len(mlphi_list)+1))
    geo_mean = 1

    for elem in mlphi_list:
        geo_mean *= elem

    geo_mean = geo_mean**(1/float(len(mlphi_list)))

    #print(minmax_list)
    mlphi_list = [None if i==0.0 else i for i in mlphi_list]

    plt.plot(indexes, mlphi_list, label = "ML-Phi", color = '#0000CC')
    if avgRRT != None:
        plt.plot(indexes, avgRRT, label = "Average RRT", color = '#808080')
    plt.plot([0,len(mlphi_list)],[geo_mean, geo_mean],'r--')
    plt.legend(loc = 4)
    plt.xlim(0,len(mlphi_list))
    plt.ylim(0,1.0)
    #plt.fill_between(num, stdevbel0,stdevabv0, color = '#808080', alpha = .1)
    plt.title(title)
    plt.xlabel("Center Of Codon Window")
    plt.ylabel("ML-Phi Windows")

    plt.show()


def add_col_to_df(Phi_list, df):
    Phi_min_list = []
    Phi_max_list = []
    Phi_avg_list = []
    Phi_median_list = []

    for l in Phi_list:
        if not l:
            Phi_min_list.append(None)
            Phi_avg_list.append(None)
            Phi_max_list.append(None)
            Phi_median_list.append(None)
        else:
            Phi_min_list.append(min(l))
            Phi_avg_list.append(sum(l) / len(l))
            Phi_max_list.append(max(l))
            Phi_median_list.append(median(l))

    #print(Phi_min_list)
    df['Phi_min'] = Phi_min_list
    df['Phi_avg'] = Phi_avg_list
    df['Phi_max'] = Phi_max_list
    df['Phi_median'] = Phi_median_list

    return df

In [2]:
import os
import helpers
PATH='/lustre/isaac/proj/UTK0196/codon-expression-data/fullTableForTrainning/'

filelist = os.listdir(PATH)
df_list = [pd.read_csv(PATH+file) for file in filelist]

s = []
for file, _df in zip(filelist, df_list):
    species = file.partition('_')[0]
    l = [species]*len(_df)
    s.extend(l)

df = pd.concat(df_list)
df['species'] = s
df = helpers.clean(df, 'Sequence')


In [3]:
[print(file) for file in filelist]

Musmusculus_fulltable.csv
Saccharomyces_Cerevisiae_s288c_fulltable.csv
celegan_fulltable.csv
atha_fulltable.csv
ecoli_fulltable.csv
Drosophila_Melanogaster_fulltable.csv
yeasts288c_fulltable.csv


[None, None, None, None, None, None, None]

In [4]:
set(df['species'] )

{'Drosophila',
 'Musmusculus',
 'Saccharomyces',
 'atha',
 'celegan',
 'ecoli',
 'yeasts288c'}

In [5]:
df_yeast = df.loc[df['species'] == 'yeasts288c']

my_windows_yeast = []
my_estimates_yeast = []
for seq in df_yeast['Sequence']:
    my_w = cal_mle_windows(seq, windowSize=9)
    my_e = calPhiForGene(seq)
    my_windows_yeast.append(my_w)
    my_estimates_yeast.append(my_e)
    
df_yeast['mlphi_est'] = my_estimates_yeast

/lustre/isaac/scratch/ababjac/pyvenv/lib/python3.9/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/lustre/isaac/scratch/ababjac/pyvenv/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_2433373/2026609916.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yeast['mlphi_est'] = my_estimates_yeast


In [6]:
df_yeast2 = df.loc[df['species'] == 'Saccharomyces']

my_windows_yeast2 = []
my_estimates_yeast2 = []
for seq in df_yeast2['Sequence']:
    my_w = cal_mle_windows(seq, windowSize=9)
    my_e = calPhiForGene(seq)
    my_windows_yeast2.append(my_w)
    my_estimates_yeast2.append(my_e)
    
df_yeast2['mlphi_est'] = my_estimates_yeast2

/tmp/ipykernel_2433373/2809636888.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yeast2['mlphi_est'] = my_estimates_yeast2


In [9]:
df_ecoli = df.loc[df['species'] == 'ecoli']

my_windows_ecoli = []
my_estimates_ecoli = []
for seq in df_ecoli['Sequence']:
    my_w = cal_mle_windows(seq, windowSize=9)
    my_e = calPhiForGene(seq)
    my_windows_ecoli.append(my_w)
    my_estimates_ecoli.append(my_e)
    
df_ecoli['mlphi_est'] = my_estimates_ecoli

/tmp/ipykernel_2433373/3535541753.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ecoli['mlphi_est'] = my_estimates_ecoli


In [11]:
df_celegan = df.loc[df['species'] == 'celegan']

my_windows_celegan = []
my_estimates_celegan = []
for seq in df_celegan['Sequence']:
    my_w = cal_mle_windows(seq, windowSize=9)
    my_e = calPhiForGene(seq)
    my_windows_celegan.append(my_w)
    my_estimates_celegan.append(my_e)
    
df_celegan['mlphi_est'] = my_estimates_celegan

/lustre/isaac/scratch/ababjac/pyvenv/lib/python3.9/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/lustre/isaac/scratch/ababjac/pyvenv/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_2433373/1232956218.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_celegan['mlphi_est'] = my_estimates_celegan


In [13]:
df_dros = df.loc[df['species'] == 'Drosophila']

my_windows_dros = []
my_estimates_dros = []
for seq in df_dros['Sequence']:
    my_w = cal_mle_windows(seq, windowSize=9)
    my_e = calPhiForGene(seq)
    my_windows_dros.append(my_w)
    my_estimates_dros.append(my_e)
    
df_dros['mlphi_est'] = my_estimates_dros

/lustre/isaac/scratch/ababjac/pyvenv/lib/python3.9/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/lustre/isaac/scratch/ababjac/pyvenv/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_2433373/1275890825.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dros['mlphi_est'] = my_estimates_dros


In [15]:
df_atha = df.loc[df['species'] == 'atha']

my_windows_atha = []
my_estimates_atha = []
for seq in df_atha['Sequence']:
    my_w = cal_mle_windows(seq, windowSize=9)
    my_e = calPhiForGene(seq)
    my_windows_atha.append(my_w)
    my_estimates_atha.append(my_e)
    
df_atha['mlphi_est'] = my_estimates_atha

/tmp/ipykernel_2433373/1075574055.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_atha['mlphi_est'] = my_estimates_atha


In [16]:
cai = pd.read_csv('./data/cai.csv')
cai = cai.reindex(cai['Unnamed: 0'])
cai['species'] = df['species']

In [17]:
cai = cai[cai['species'] != 'Musmusculus']

In [18]:
cai

,Unnamed: 0,GeneName,median_exp,Sequence,CAI,codons_cleaned,species
Unnamed: 0,,,,,,,
0,0,Gnai3,3.555348,ATGGGCTGCACGTTGAGCGCCGAGGACAAGGCGGCGGTGGAGCGGA...,0.741979,ATG GGC TGC ACG TTG AGC GCC GAG GAC AAG GCG GC...,Saccharomyces
1,1,Cdc45,0.371564,ATGTTCGTGACCGATTTCCGCAAGGAGTTCTACGAGACGGTCCACA...,0.769123,ATG TTC GTG ACC GAT TTC CGC AAG GAG TTC TAC GA...,Saccharomyces
2,2,Scml2,0.000000,ATGGCAGAGCCTGCCACTGGAGTGCAACTTGCTGGTTCTGGAGAGC...,0.726407,ATG GCA GAG CCT GCC ACT GGA GTG CAA CTT GCT GG...,Saccharomyces
3,3,Apoh,-2.302585,ATGGTTTCCCCGGTGCTCGCCTTGTTCTCCGCCTTCCTCTGCCATG...,0.768817,ATG GTT TCC CCG GTG CTC GCC TTG TTC TCC GCC TT...,Saccharomyces
4,4,Narf,2.302585,ATGAAGTGTGAGCACTGCACACGAAAGGAATGTAGTAAAAAATCAA...,0.790900,ATG AAG TGT GAG CAC TGC ACA CGA AAG GAA TGT AG...,Saccharomyces
...,...,...,...,...,...,...,...
5031,5031,Trip12,2.564949,ATGTCCAACCGGCCTAATAACAATCCAGGGGGGTCACTGCGACGTT...,0.699154,ATG TCC AAC CGG CCT AAT AAC AAT CCA GGG GGG TC...,yeasts288c
5032,5032,Slc16a14,0.587787,ATGTATACGAGTCATGAAGATATCGGGTATGACCTTGAAGACGACC...,0.753685,ATG TAT ACG AGT CAT GAA GAT ATC GGG TAT GAC CT...,yeasts288c
5033,5033,Sp100,-1.203973,ATGGAAGGGAGCGACGGCAGCCCCAGGATGTCCACTGAACAGGAGA...,0.749642,ATG GAA GGG AGC GAC GGC AGC CCC AGG ATG TCC AC...,yeasts288c


In [19]:
new_df = pd.concat([df_yeast2, df_celegan, df_atha, df_ecoli, df_dros, df_yeast])

In [20]:
new_df['CAI'] = cai['CAI']

In [22]:
new_df.to_csv('./data/cub.csv')